In [1]:
import os
import PyPDF2
import markdown
from transformers import pipeline
import pandas as pd

In [2]:
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
DATA_FOLDER = "data/dataset/"
OUTPUT_FOLDER = "data/processed_data"

def extract_pdf_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text

def extract_md_text(md_path):
    with open(md_path, "r", encoding="utf-8") as f:
        md_text = f.read()
    return markdown.markdown(md_text)  # Convert MD to plain text if needed

In [4]:
# Process all files
data_dir = "data/dataset/"
all_text = []

for file in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file)
    if file.endswith(".pdf"):
        all_text.append(extract_pdf_text(file_path))
    elif file.endswith(".md"):
        all_text.append(extract_md_text(file_path))

# Split into chunks (adjust chunk_size as needed)
chunk_size = 512  # characters
chunks = [text[i:i+chunk_size] for text in all_text for i in range(0, len(text), chunk_size)]

In [5]:
import torch
print(torch.cuda.is_available())  # Should output True

True


In [6]:
model_name = "Qwen/Qwen2.5-3B-Instruct"
pipe = pipeline("text-generation", model=model_name, device_map="cuda")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [8]:
import numpy as np

target_pairs = 500  # <--- SET YOUR DESIRED NUMBER HERE
qa_pairs = []
current_count = 0

while current_count < target_pairs:
    for chunk in chunks:

        prompt = f"""
        Generate **ONE** question and answer based on this text.
        Return ONLY in format: "Q: [question]\nA: [answer]".
        Text: {chunk[:1000]}
        """

        response = pipe(
            prompt,
            max_new_tokens=256,
            temperature=0.3,
        )[0]['generated_text']

        try:
            q, a = response.split("A: ")[0].strip(), response.split("A: ")[1].strip()
            qa_pairs.append({"question": q.replace("Q: ", ""), "answer": a, "context": chunk})
            print (f"Generated QA pair {current_count + 1}")
            current_count += 1
            if current_count >= target_pairs:
                break  # Stop when target is reached
        except:
            continue  # Skip errors

    # Optional: Shuffle chunks to avoid bias from looping
    np.random.shuffle(chunks)

# Convert to DataFrame
df = pd.DataFrame(qa_pairs)
print(f"Generated {len(df)} QA pairs")  # Verify count

Generated QA pair 1
Generated QA pair 2


KeyboardInterrupt: 

In [9]:
import openai

target_pairs = 50  # Set desired number
qa_pairs = []
current_count = 0

for chunk in chunks:
    if current_count >= target_pairs:
        break

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4"
        messages=[
            {"role": "system", "content": "Generate ONE question and answer from technical text. Use format: Q: [question]\nA: [answer]"},
            {"role": "user", "content": f"Text: {chunk[:1000]}"}
        ],
        temperature=0.2,  # Lower for factual accuracy
        max_tokens=256
    )

    try:
        full_response = response.choices[0].message.content
        q, a = full_response.split("A: ")
        q = q.replace("Q: ", "").strip()
        qa_pairs.append({"question": q, "answer": a.strip(), "context": chunk})
        current_count += 1
        print (f"Generated QA pair {current_count}")
    except:
        continue  # Skip malformed responses

# Convert to DataFrame
df = pd.DataFrame(qa_pairs)
print(f"Generated {len(df)} QA pairs")

Generated QA pair 1
Generated QA pair 2
Generated QA pair 3
Generated QA pair 4
Generated QA pair 5
Generated QA pair 6
Generated QA pair 7
Generated QA pair 8
Generated QA pair 9
Generated QA pair 10
Generated QA pair 11
Generated QA pair 12
Generated QA pair 13
Generated QA pair 14
Generated QA pair 15
Generated QA pair 16
Generated QA pair 17
Generated QA pair 18
Generated QA pair 19
Generated QA pair 20
Generated QA pair 21
Generated QA pair 22
Generated QA pair 23
Generated QA pair 24
Generated QA pair 25
Generated QA pair 26
Generated QA pair 27
Generated QA pair 28
Generated QA pair 29
Generated QA pair 30
Generated QA pair 31
Generated QA pair 32
Generated QA pair 33
Generated QA pair 34
Generated QA pair 35
Generated QA pair 36
Generated QA pair 37
Generated QA pair 38
Generated QA pair 39
Generated QA pair 40
Generated QA pair 41
Generated QA pair 42
Generated QA pair 43
Generated QA pair 44
Generated QA pair 45
Generated QA pair 46
Generated QA pair 47
Generated QA pair 48
G

In [12]:
from ragas.metrics import answer_relevancy, faithfulness
from ragas import evaluate
from datasets import Dataset

dataset = Dataset.from_pandas(df)
score = evaluate(dataset, metrics=[answer_relevancy])
df = df[score["answer_relevancy"] > 0.7]
dataset.save_to_disk("qa_dataset")

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

TypeError: '>' not supported between instances of 'list' and 'float'

In [ ]:
train_df, val_df, test_df = np.split(df.sample(frac=1), [0.8, 0.1, 0.1])

#### Synethetic Data Generation

In [42]:
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer()
document_paths = [os.path.join(OUTPUT_FOLDER, fname) for fname in os.listdir(OUTPUT_FOLDER)]
synthesizer.generate_goldens_from_docs(document_paths=document_paths)

# Access the synthetic data
synthetic_data = synthesizer.synthetic_goldens


Event loop is already running. Applying nest_asyncio patch to allow async execution...


ValueError: Unsupported file format: .json

In [ ]:
combined_dataset = []

# Add extracted texts
for filename, text in extracted_texts.items():
    combined_dataset.append({'source': filename, 'text': text, 'synthetic': False})

# Add synthetic data
for item in synthetic_data:
    combined_dataset.append({'source': 'synthetic', 'text': item['input'], 'synthetic': True})

# Example: Display the first entry
print(combined_dataset[0])